In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')
#pd.pandas.set_option('display.max_columns', None)

In [2]:
import os
os.getcwd()

'D:\\faizan\\DSP\\Python\\Capstone Project'

In [47]:
dataset = pd.read_excel("Win_Prediction_Data-Copy.xlsx")
dataset.head()

,Client Category,Solution Type,Deal Date,Sector,Location,VP Name,Manager Name,Deal Cost,Deal Status Code
0,Telecom,Solution 7,2012-03-27,Sector 24,L5,Ekta Zutshi,Gopa Trilochana,150000.00,Won
1,Telecom,Solution 7,2012-09-25,Sector 24,L5,Ekta Zutshi,Gopa Trilochana,744705.88,Won
2,Internal,Solution 59,2011-08-01,Sector 20,Others,Ekta Zutshi,Russell Dahlen,60000.00,Lost
3,Internal,Solution 59,2011-04-28,Sector 20,Others,Ekta Zutshi,Russell Dahlen,60000.00,Lost
4,Internal,Solution 32,2011-06-03,Sector 20,Others,Ekta Zutshi,Russell Dahlen,80882.35,Lost


In [48]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [49]:
dataset.head()

,Client Category,Solution Type,Deal Date,Sector,Location,VP Name,Manager Name,Deal Cost,Deal Status Code
0,Telecom,Solution 7,2012-03-27,Sector 24,L5,Ekta Zutshi,Gopa Trilochana,150000.00,Won
1,Telecom,Solution 7,2012-09-25,Sector 24,L5,Ekta Zutshi,Gopa Trilochana,744705.88,Won
2,Internal,Solution 59,2011-08-01,Sector 20,Others,Ekta Zutshi,Russell Dahlen,60000.00,Lost
3,Internal,Solution 59,2011-04-28,Sector 20,Others,Ekta Zutshi,Russell Dahlen,60000.00,Lost
4,Internal,Solution 32,2011-06-03,Sector 20,Others,Ekta Zutshi,Russell Dahlen,80882.35,Lost


In [50]:
dataset.drop_duplicates(keep='first').shape

(10047, 9)

In [51]:
X = dataset[['Client Category', 'VP Name', 'Manager Name', 'Deal Cost']]
y = dataset[['Deal Status Code']]

In [53]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y['Deal Status Code'] = le.fit_transform(y['Deal Status Code'])
y.head()

,Deal Status Code
0,1
1,1
2,0
3,0
4,0


In [54]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=1)

X_train.shape, X_test.shape

((7042, 4), (3019, 4))

In [55]:
X_train.head()

,Client Category,VP Name,Manager Name,Deal Cost
217,Others,Sargar Deep Rao,Tanuj Bindra,0.00
9619,Finance,Hardeep Suksma,Atul Chaudhary,600000.00
5142,Others,Long Bergstrom,Jewell Tunstall,398529.41
8189,Automobiles,Visvajeet Das,Gunjan Singh,1176470.59
248,Domestic Private Bank,Mervin Harwood,Prateek Dhotre,132352.94


In [95]:
# imputation transformer
trf1 = ColumnTransformer([
    ('Client Category',SimpleImputer(strategy='most_frequent'),[0])
],remainder='passthrough')

In [86]:
# one hot encoding
trf2 = ColumnTransformer([
    ('Client Category',OneHotEncoder(drop='first', sparse=False),[0]),
    ('VP Name',OneHotEncoder(drop='first', sparse=False),[1]),
    ('Manager Name',OneHotEncoder(drop='first', sparse=False),[2])
],remainder='passthrough')

In [127]:
# one hot encoding
trf2 = ColumnTransformer([
    ('OHE',OneHotEncoder(sparse=False, handle_unknown='ignore'),[0,1,2])
],remainder='passthrough')

In [128]:
# Scaling
trf3 = ColumnTransformer([
    ('scale',StandardScaler(),slice(0,359))
])

In [129]:
# train the model
trf4 = RandomForestClassifier(n_estimators=500)

In [130]:
pipe = make_pipeline(trf1,trf2,trf3,trf4)

In [131]:
# train
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Client Category',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [0])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('OHE',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [0, 1, 2])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', StandardScaler(),
                                                  slice(0, 359, None))])),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=500

In [132]:
y_pred = pipe.predict(X_test)

In [133]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7959589267969527

In [134]:

# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.7725099199948384

In [135]:
# export 
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

In [136]:
# pickling the model
import pickle
pickle_out = open("pipe.pkl", "wb")
pickle.dump(pipe, pickle_out)
pickle_out.close()


In [120]:
dataset['Client Category'].nunique()

41

In [115]:
dataset['VP Name'].nunique()

43

In [116]:
dataset['Manager Name'].nunique()

278

In [118]:
278+41+43-3

359